# Fabric SDK Python

Before getting started, make sure the Python SDK is installed properly and the network is up and running.
For more information regarding installation process and how to set up the network please refer to [this](https://github.com/hyperledger/fabric-sdk-py/blob/main/docs/source/tutorial.md).

## Prepare a Testing Environment

### Install Fabric SDK

```bash
$ git clone https://github.com/hyperledger/fabric-sdk-py.git
$ cd fabric-sdk-py
$ make install
```

### Setup a Fabric Network

If you already have a running fabric network, ignore this.

To start an example fabric network you can simply run the following command:

```bash
$ HLF_VERSION=1.4.6
$ docker pull hyperledger/fabric-peer:${HLF_VERSION}
$ docker pull hyperledger/fabric-orderer:${HLF_VERSION}
$ docker pull hyperledger/fabric-ca:${HLF_VERSION}
$ docker pull hyperledger/fabric-ccenv:${HLF_VERSION}
$ docker-compose -f test/fixtures/docker-compose-2orgs-4peers-tls.yaml up
```

Run the bellow code block to import an instance of a client from the network profile already provided

In [1]:
from hfc.fabric import Client

cli = Client(net_profile="test/fixtures/network.json")

print(cli.organizations)  # orgs in the network
print(cli.peers)  # peers in the network
print(cli.orderers)  # orderers in the network
print(cli.CAs)  # ca nodes in the network

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


{'orderer.example.com': <hfc.fabric.organization.Organization object at 0x7fbb95a5ef60>, 'org1.example.com': <hfc.fabric.organization.Organization object at 0x7fbb977e90f0>, 'org2.example.com': <hfc.fabric.organization.Organization object at 0x7fbb95938588>}
{'peer0.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fbb959386a0>, 'peer1.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fbb95938dd8>, 'peer0.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fbb95938a90>, 'peer1.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fbb95938860>}
{'orderer.example.com': <hfc.fabric.orderer.Orderer object at 0x7fbb95938978>}
{'ca-org1': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fbb95938828>, 'ca-org2': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fbb959389b0>}


In [2]:
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin') # User instance with the Org1 admin's certs
org2_admin = cli.get_user(org_name='org2.example.com', name='Admin') # User instance with the Org2 admin's certs
orderer_admin = cli.get_user(org_name='orderer.example.com', name='Admin') # User instance with the orderer's certs

Run the bellow code snippet to create a channel with the name 'business' channel.
This command will return True if the channel was created successfully.

In [3]:
# Create a New Channel, the response should be true if succeed
response = await cli.channel_create(
                    orderer='orderer.example.com',
                    channel_name='businesschannel',
                    requestor=org1_admin,
                    config_yaml='test/fixtures/e2e_cli/',
                    channel_profile='TwoOrgsChannel'
                    )

# response = true is returned if the channel is created successfully
print(response)

FABRIC_CFG_PATH set to /home/rohan/Documents/fabric-sdk-py/test/fixtures/e2e_cli/
Configtx file successfully created in current             directory
{'tx_id': '3595d96aa2fda8c12e225fbd4689bf8e511cda14c742a27887fa94321aee9686', 'nonce': b'\x94\x13\xc3\xd9\xb3\xba\xe7V\xb8Z\xd3\xad\x9b\xb9<\xbe\xd1\xc8Y\xd9{\xe1d\xd5', 'signatures': [b"\n\xd3\x06\n\xb6\x06\n\x07Org1MSP\x12\xaa\x06-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMNU2FuIEZy\nYW5jaXNjbzEZMBcGA1UEChMQb3JnMS5leGFtcGxlLmNvbTEcMBoGA1UEAxMTY2Eu\nb3JnMS5leGFtcGxlLmNvbTAeFw0xODEwMTkwMzQ4MDBaFw0yODEwMTYwMzQ4MDBa\nMGwxCzAJBgNVBAYTAlVTMRMwEQYDVQQIEwpDYWxpZm9ybmlhMRYwFAYDVQQHEw1T\nYW4gRnJhbmNpc2NvMQ8wDQYDVQQLEwZjbGllbnQxHzAdBgNVBAMMFkFkbWluQG9y\nZzEuZXhhbXBsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAR2zFPDBYIy\njZSXaB15ILW9MRUYkSiksD6Io+VBbsJex2S7Do7lJhYfpyg5Z1LEHLCQgHH/VJ1F\nMZmNHgIIHqWPo00wSzAOBgNVHQ8BAf8EBAMCB4AwDAYDVR0TAQH/BAIwADArB

True


Run the below code snippet to make the peers of Org1 join the channel.

In [4]:
# Join Peers into Channel, the response should be true if succeed
responses = await cli.channel_join(
                       requestor=org1_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

get genesis block successfully, block=data_hash: "\217\232\252\311\240\204\332\245\203\203\313\263$\377A\234\037[:X\300>\246\005\266\227\322\020\244O4\217"



True


Run the below code snippet to make the peers of Org1 join the channel.

In [5]:
# For operations on peers from org2.example.com, org2_admin is required as requestor
responses = await cli.channel_join(
                       requestor=org2_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org2.example.com',
                      'peer1.org2.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

get genesis block successfully, block=data_hash: "\217\232\252\311\240\204\332\245\203\203\313\263$\377A\234\037[:X\300>\246\005\266\227\322\020\244O4\217"



True


Run the below code snippet to install an example chaincode which can be found at `test/fixtures/chaincode` on the peers of org1.

In [ ]:
# This installs the example chaincode on the peers
# Make the client know there is a channel in the network
cli.new_channel('businesschannel')

#The GOPTAH settings is required for only running the example chaincode in the SDK
import os
gopath_bak = os.environ.get('GOPATH', '')
gopath = os.path.normpath(os.path.join(
                      os.path.dirname(os.path.realpath('__file__')),
                      'test/fixtures/chaincode'
                     ))

# The response should be true if succeed
responses = loop.run_until_complete(cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/example_cc',
               cc_name='example_cc',
               cc_version='v1.0'
               ))


Run the below code snippet to instantiate the installed chaincode.
During instantiation you should provide the endorsement policy for the chaincode.
You can find more details regarding the endorsement policy [here](https://hyperledger-fabric.readthedocs.io/en/release-1.4/endorsement-policies.html)

In [ ]:
# Instantiate Chaincode in Channel, the response should be true if succeed
args = ['a', '200', 'b', '300']

# This policy specifies the endorsement policy which is required while instantiating the chaincode
policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0},
        ]
    }
}
response = loop.run_until_complete(cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               ))

Run the below code snippet to invoke the installed chaincode with the arguments as mentioned in args.

In [ ]:
# Invoke a chaincode
args = ['a', 'b', '100']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger, default is on tx event
               #cc_pattern='^invoked*' # if you want to wait for chaincode event and you have a `stub.SetEvent("invoked", value)` in your chaincode
               ))


In [ ]:
# Query a chaincode
args = ['b']
# The response should be true if succeed
response = loop.run_until_complete(cli.chaincode_query(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc'
               ))